In [11]:
## Import Libraries
import json
import csv
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy.spatial.distance
from copy import deepcopy

In [12]:
##Import data
qa = np.genfromtxt("uid_qa.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U128'),])

In [13]:
fe = np.genfromtxt("uid_pre_elim.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U256'),])

In [14]:
fe.dtype.names = ('uniqueID', 'bn', 'qn', 'pre_que', 'curr_elim')

#fe.dtype.names
#fe[971]

In [15]:
ftd = np.genfromtxt("face_id_descr.txt", delimiter = ";", skip_header = 1 , usecols = np.arange(0,2), dtype = [('U16'), ('U2056')])

In [16]:
ftd.dtype.names = ('img_id', 'description')
len(ftd['img_id'])
len(set(ftd['img_id']))

193

In [17]:
## Function- to convert sring into list.
## First convert strings to list of rows equal to rows in fe by making a function
## All image ids are one string in each row. By converting this string into list we will make each image ID an item in the list.

def mk_list(str_vector):
    des_byid = str_vector
    all_x = []
    for i in des_byid:
        j = i.split()
        #if len(j) > 0:
        all_x.append(j)

    return all_x


In [18]:
## convert strings in each row of pre-que to list of itmens.
## each row of all_x contains a list of image id(S) before question is asked and action like elimination/selection is done.

all_x = mk_list(fe['pre_que'])
len(all_x)
all_x[972]
len(all_x[0])
print(len(qa['que'][1473]))

29


In [19]:
##check blanks
## As there are some rows where there is only one image or blank/erronous question. We check that and save those indices.
## create blank list
#good_index = ['True']*len(all_x)

bl =[]

for img,q in zip(enumerate(all_x), mk_list(qa['que'])):
    if (len(img[1]) <2 or len(q) < 2) and img[0] not in bl:
        bl.append(img[0])
print(bl)


[10, 43, 114, 115, 117, 118, 130, 131, 134, 135, 136, 137, 138, 139, 140, 234, 358, 408, 537, 607, 636, 674, 923, 928, 929, 931, 932, 949, 1078, 1079, 1196, 1250, 1251, 1258, 1264, 1274, 1364, 1433, 1493, 1494, 1545, 1600, 1753, 1907, 1996, 2045, 2077, 2095, 2103, 2105, 2106, 2167, 2168, 2169, 2201, 2308, 2312, 2413, 2455, 2535, 2536, 2542, 2657, 2681, 2706, 2772, 2773, 2774, 2778, 2779, 2781, 2782, 2783, 2785, 2786, 2787, 2788, 2789, 2790, 2791, 2798, 2873, 2876, 2902, 2942, 2949, 2965, 3079, 3201, 3207, 3258, 3308, 3411, 3416, 3426, 3439, 3440, 3444, 3445, 3446, 3449, 3496, 3506, 3507, 3549, 3600, 3602, 3610, 3612, 3616, 3619, 3621, 3624, 3626, 3628, 3652, 3654, 3655, 3656, 3665, 3682, 3715, 3726, 3727, 3730, 3733, 3735, 3829, 3912, 3954, 3959, 4001, 4002, 4003, 4004, 4005, 4006, 4007, 4008, 4024, 4025, 4302, 4335, 4363, 4415, 4450, 4522, 4523, 4536, 4572, 4621, 4723, 4734, 4736, 4737, 4792, 4793, 4794, 4823, 4824, 4825, 4826, 4827, 4828, 4829, 4830, 4898, 4904, 4916, 4989, 5019, 504

In [20]:
len(all_x)
len(mk_list(qa['que']))

15131

In [21]:
#print bl
#for n in bl[2000:400]:
#    print(all_x[n], mk_list(qa['que'])[n])

In [22]:
## Now for each row in the list above, find description and add it
all_i_txt1 = []
original_ind = []
bl_ind = []

for rown in range(len(all_x)):
    if rown in bl:
        bl_ind.append(rown)
    else:
        #print(all_x.index)
        int_i_txt = []
        for img in all_x[rown]:
            if img in ftd['img_id']:
                d = str(ftd['description'][list(ftd['img_id']).index(img)])
                int_i_txt.append(str(d))
        original_ind.append(rown)
        all_i_txt1.append(int_i_txt)
        
            
            

In [23]:
#print check
#print(type(all_i_txt[0]))
#print(len(all_i_txt))
#print(all_i_txt[971])
#print(all_i_txt[1473])
print(len(original_ind))
print(len(all_i_txt1))
print(len(bl_ind))
print(len(bl))
print(bl == bl_ind)
#print(original_ind)
#for it in original_ind:
#    print(qa['ans'][it])

14644
14644
487
487
True


In [24]:
## add question and answer to each data point

all_i_txt = [row + [qa['que'][rn].strip(), qa['ans'][rn].strip()] for rn, row in zip(original_ind, all_i_txt1)]

    
    
    

In [25]:
type(all_i_txt[0])
all_i_txt[0][15:18]

['An asian man in his early twenties.  He has a youthful face with some acne.  He has mid length black hair with shaved sides.  He has tanned skin.. 24 years. asian. Hispanic male[comma] mid twenties,short black hair, average build. 24 years. hispanic. A youngish man[comma] regular features, average looking. The hair parting is on the other side, gives a different look to the face, person seems to have some acne or skin problems. . 21 years. Asian. male. 20 years. asian. Mouth and ears look pretty normal.  Nose is a bit bulbous and red near the bottom.  Looks a bit angry because of the narrow eyes.  Bushy dark eyebrows.  Hair is longer on the top of the head than on the sides.. 28 years. african. . Chubby face and acne on face detected as well. Physically they appear to have broad shoulders with good posture.. 24 years. latino. . The person is average looking and simple.Small eyes with straight hair[comma]broad face.. 23 years. african. . Medium height[comma] Asian guy[comma] has acne 

In [ ]:
## check addition of above objects
for i in range(len(all_i_txt)):
    print(all_i_txt[i][-2:])

In [26]:
#print(all_i_txt[0])
##basic model
cv = CountVectorizer()

In [27]:
## all similarities
all_sim = []
for l in range(len(all_i_txt)):
    H = cv.fit_transform(all_i_txt[l])
    sim = cosine_similarity(H)
    all_sim.append(sim)

In [28]:
#similarities[-2] meaning taking only the array for question
all_sim_q = []
for i in range(len(all_sim)):
    all_sim_q.append(all_sim[i][-2][:-2])
print(len(all_sim_q))
#print(all_sim_q[971])
#print(len(all_sim))

14644


In [ ]:
## Sort all the above similarities with argsort which gives indices
## or we should not take top5 and make a threshold above which we will include it in the list of output?
sorted_ind = []
for si in all_sim_q:
    y_top = si.argsort()[::-1]
    sorted_ind.append(y_top)
#sorted_ind[971]

In [ ]:

## with above indices, create new sorted array per que, this is done to then find max step difference 
sorted_sim_all = []
for q,w in zip(sorted_ind, all_sim_q):
    sorted_sim = []
    for e in q:
        sorted_sim.append(w[e])
    sorted_sim_all.append(sorted_sim)
print(len(sorted_sim_all))

In [ ]:
##find the difference between two similarity score in each array.
sim_dif_all = []

for d in range(len(sorted_sim_all)):
    sim_dif = [sorted_sim_all[d][f-1] - sorted_sim_all[d][f] for f in range(1, len(sorted_sim_all[d]))]
    sim_dif_all.append(sim_dif)
        
len(sim_dif_all)





#print(sim_dif_all)
##this number is the index number of max difference and will can be used in sorted_sim_all to cut off, then use 
##[:this num+1]
## the problem is that here for example if the image description is not there before the question or 
##images are not present
## befor question, the description is empty. The similarity measure with question and answer is then

In [ ]:
## The problem lies here...
##question is why do I still have cases with one image id? Even though I am removing them earlier. What am I missing?

max_dif = []
max_dif_ind =[]
for g in range(len(sim_dif_all)):
    
    maxi= max(sim_dif_all[g])
    max_dif.append(maxi)
    max_dif_ind.append(sim_dif_all[g].index(maxi))
#print(len(max_dif))
#print(len(max_dif_ind))
#print(max_dif.index(None))
#print(max_dif)
#sim_dif_all[971]
#sorted_sim_all[971]

In [ ]:
print(len(max_dif_ind))

In [ ]:
sorted_ind_lim = [indi[0:max_indi+1] for indi,max_indi in zip(sorted_ind, max_dif_ind)]


In [ ]:
##clean_all_x
all_x_clean = [all_x[i] for i in original_ind]


In [ ]:
len(all_x_clean)

In [ ]:
top_img = zip(all_x_clean, sorted_ind_lim)

In [ ]:
#top_img = zip(all_x, y_top_all)

In [ ]:
## convert to img_id from index to match with actual image ids selcted by participant
def mk_imgid(arr):
    y_pred = []
    for i,j in arr:
        yp = []
        for s in j:
            yp.append(i[s])
        y_pred.append(yp)
    return y_pred

In [ ]:
pred_img = mk_imgid(top_img)

In [ ]:
len(pred_img)
pred_img[0]

In [ ]:
all_cheat1 = mk_list(fe['curr_elim'])
all_cheat = [all_cheat1[i] for i in original_ind]

In [ ]:
len(all_cheat)
all_cheat[0]

In [ ]:
pre_all= [len(set(a).intersection(set(p)))/len(set(p)) if len(p) > 0 else 0 for a,p in zip(all_cheat, pred_img)]
rec_all = [len(set(a).intersection(set(p)))/len(set(a)) if len(a) > 0 else 0 for a,p in zip(all_cheat, pred_img)]



In [ ]:
print(len(pre_all))
print(len(rec_all))

In [ ]:
#(0.5*0.5 + 1)*pre_all[0]*rec_all[0]/ (0.5*0.5*pre_all[0] + rec_all[0])

In [ ]:
fs_all= [(0.5*0.5 + 1)*pr*re/ (0.5*0.5*pr + re) if pr > 0 or re >0 else 0 for pr, re in zip(pre_all, rec_all)]

In [ ]:
len(fs_all)

In [ ]:
pre_mean = np.mean(np.array(pre_all))
rec_mean = np.mean(np.array(rec_all))
fs_mean = np.mean(np.array(fs_all))

In [ ]:
pre_mean

In [ ]:
op1 = ["1", "CV", "default", "Cosinesimilarity and ranking difference",str(round(pre_mean,2)), str(round(rec_mean,2)), str(round(fs_mean,2))]

In [ ]:
#with open('Results with description.txt', 'w') as f:
#    f.write("No.; Model; Parameter; Decision Rules; Precision; Recall; F0.5"+ '\n')

In [ ]:
with open('Results with description.txt', 'a') as f:
    f.write('; '.join(op1) + '\n')